In [1]:
import pandas as pd
import os
import sys
from datetime import datetime
from sqlalchemy import text

# --- Configuração dos Módulos ---
sys.path.insert(0, r'C:\Scripts\modules_azure\database')
sys.path.insert(0, r'C:\Scripts\modules_azure\parameters')

from connection_azure import Connect
from azure_loader import AzureLoader
from parametros import Parametros

# --- Configurações Globais ---
SCHEMA_DEFAULT = "dbo"
NOME_TABELA = "pl_historico_diario"
DIRETORIO_RELATORIOS = r"C:\Scripts\relatórios"

In [2]:
try:
    db = Connect()
    conn = db.connect_techdb()
    
    # Query otimizada para pegar apenas o valor máximo
    query = text(f'SELECT MAX("Data") FROM {SCHEMA_DEFAULT}."{NOME_TABELA}"')
    result = conn.execute(query).scalar()
    
    if result:
        dt_max = pd.to_datetime(result)
        print(f"   -> Última data no banco: {dt_max.strftime('%d/%m/%Y')}")
    else:
        dt_max = pd.to_datetime("2020-01-01") # Data antiga padrão caso a tabela esteja vazia
        print("   -> Tabela vazia ou sem datas. Iniciando carga total.")
        
    conn.close()
    
except Exception as e:
    print(f"Erro ao consultar banco: {e}")
    sys.exit()

   -> Última data no banco: 02/02/2026


In [3]:
pastas_para_processar = []

if os.path.exists(DIRETORIO_RELATORIOS):
    # Itera sobre todas as pastas na raiz
    for nome_pasta in os.listdir(DIRETORIO_RELATORIOS):
        caminho_completo = os.path.join(DIRETORIO_RELATORIOS, nome_pasta)
        
        if os.path.isdir(caminho_completo):
            try:
                # Tenta converter o nome da pasta em data
                data_pasta = datetime.strptime(nome_pasta, "%d-%m-%Y")
                
                # Se a data da pasta for MAIOR que a última do banco, adiciona na fila
                if data_pasta > dt_max:
                    pastas_para_processar.append({
                        "data": data_pasta,
                        "caminho": caminho_completo,
                        "dia_str": nome_pasta
                    })
            except ValueError:
                continue # Ignora pastas que não são datas

    # Ordena cronologicamente para inserir na ordem correta
    pastas_para_processar.sort(key=lambda x: x["data"])
else:
    print(f"Erro: Diretório {DIRETORIO_RELATORIOS} não encontrado.")
    sys.exit()

if not pastas_para_processar:
    print("   -> O banco já está atualizado com a data mais recente disponível.")
    sys.exit()

print(f"   -> Encontrados {len(pastas_para_processar)} dias novos para processar.")

   -> Encontrados 4 dias novos para processar.


In [4]:
dfs_para_upload = []

for item in pastas_para_processar:
    dia_str = item['dia_str']
    caminho_dia = item['caminho']
    
    try:
        # Busca subpastas (horários)
        subpastas = [f for f in os.listdir(caminho_dia) if os.path.isdir(os.path.join(caminho_dia, f))]
        
        if not subpastas:
            print(f"   [Pular] {dia_str}: Nenhuma subpasta encontrada.")
            continue
            
        # Pega a primeira pasta de horário
        pasta_horario = subpastas[0]
        caminho_arquivo = os.path.join(caminho_dia, pasta_horario, "base_btg.xlsx")
        
        if not os.path.exists(caminho_arquivo):
            print(f"   [Pular] {dia_str}: Arquivo base_btg.xlsx não encontrado.")
            continue

        # Leitura do Excel
        try:
            df = pd.read_excel(caminho_arquivo, header=2)
            if "Conta" not in df.columns:
                df = pd.read_excel(caminho_arquivo)
        except:
            df = pd.read_excel(caminho_arquivo)

        # Limpeza de nomes de colunas
        df.columns = [str(c).replace(" (R$)", "").strip() for c in df.columns]
        
        # Seleção de Colunas
        colunas_desejadas = [
            "Conta", "Assessor", "PL Total", "Tipo", "Conta Corrente", 
            "Fundos", "Renda Fixa", "Renda Variável", "Previdência", "Derivativos"
        ]
        
        # Filtra apenas as que existem no arquivo
        cols_existentes = [c for c in colunas_desejadas if c in df.columns]
        df_final = df[cols_existentes].copy()
        
        # Adiciona colunas de Data
        df_final['Data'] = item['data'] # Objeto datetime
        df_final['Mês'] = item['data'].strftime("%Y-%m")
        
        # Tratamento de Tipos
        if 'Conta' in df_final.columns:
            df_final['Conta'] = df_final['Conta'].astype(str).str.strip()
            
        # Preenche nulos numéricos com 0
        cols_numericas = ["PL Total", "Conta Corrente", "Fundos", "Renda Fixa", "Renda Variável", "Previdência", "Derivativos"]
        for col in cols_numericas:
            if col in df_final.columns:
                df_final[col] = pd.to_numeric(df_final[col], errors='coerce').fillna(0)

        dfs_para_upload.append(df_final)
        print(f"   [OK] {dia_str}: {len(df_final)} registros processados.")

    except Exception as e:
        print(f"   [ERRO] Falha ao ler dia {dia_str}: {e}")

   [OK] 03-02-2026: 1196 registros processados.
   [OK] 04-02-2026: 1105 registros processados.
   [OK] 05-02-2026: 1186 registros processados.
   [OK] 06-02-2026: 1187 registros processados.


In [5]:
if dfs_para_upload:
    df_consolidado = pd.concat(dfs_para_upload, ignore_index=True)
    
    # Tratamento final de nomes para Assessor (Opcional, mas recomendado para padronização)
    if 'Assessor' in df_consolidado.columns:
        df_consolidado['Assessor'] = df_consolidado['Assessor'].astype(str).str.upper()
    
    AzureLoader.enviar_df(
        df=df_consolidado,
        nome_tabela=NOME_TABELA,
        if_exists='append', # IMPORTANTE: Append para manter o histórico
        schema=SCHEMA_DEFAULT
    )
else:
    print("   -> Nenhum dado válido foi gerado para upload.")

[AzureLoader] Subindo tabela: dbo.pl_historico_diario (4674 linhas)...
[AzureLoader] Chunksize calculado: 174 linhas/lote (Colunas: 12).
[AzureLoader] Concluido: pl_historico_diario atualizada.
